# IMPORTS

In [ ]:
import os, sys
import torch
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import CL_inference as cl_inference

%load_ext autoreload

%matplotlib notebook
plt.style.use('default')
plt.close('all')

font, rcnew = cl_inference.plot_utils.matplotlib_default_config()
mpl.rc('font', **font)
plt.rcParams.update(rcnew)
plt.style.use('tableau-colorblind10')
%config InlineBackend.figure_format = 'retina'

N_threads = cl_inference.train_tools.set_N_threads_(N_threads=1)
device = cl_inference.train_tools.set_torch_device_()

In [ ]:
path_global_load = '/cosmos_storage/dlopez/Projects/CL_inference/models'

In [ ]:
list_threshold_bias = [1., 3.]

y_final_all = {}
bin_centers_all = {}
fraction_biased_all = {}
NN_points_all = {}

y_final_noCL = {}
bin_centers_final_noCL = {}
fraction_biased_noCL = {}
NN_points_noCL = {}

y_final_CL = {}
bin_centers_final_CL = {}
fraction_biased_CL = {}
NN_points_CL = {}


for ii_threshold, threshold_bias in enumerate(list_threshold_bias):

    y_final_all[threshold_bias] = np.load(
        os.path.join(path_global_load, "only_inference_models_all_kmax_0.6_box_3000", "y_final.npy")
    )
    bin_centers_all[threshold_bias] = np.load(
        os.path.join(path_global_load, "only_inference_models_all_kmax_0.6_box_3000", "bin_centers_final.npy")
    )
    NN_points_all[threshold_bias] = np.load(
        os.path.join(path_global_load, "only_inference_models_all_kmax_0.6_box_3000", "NN_points_final.npy")
    )
    fraction_biased_all[threshold_bias] = np.zeros((y_final_all[threshold_bias].shape[0], y_final_all[threshold_bias].shape[1]))
    mask =np.abs(bin_centers_all[threshold_bias]) > 2
    for ii in range(y_final_all[threshold_bias].shape[0]):
        for jj in range(y_final_all[threshold_bias].shape[1]):
            fraction_biased_all[threshold_bias][ii,jj] = np.sum(y_final_all[threshold_bias][ii, jj][mask[ii, jj]])/NN_points_all[threshold_bias][ii,jj]



    y_final_noCL[threshold_bias] = {}
    bin_centers_final_noCL[threshold_bias] = {}
    fraction_biased_noCL[threshold_bias] = {}
    NN_points_noCL[threshold_bias] = {}
    
    y_final_CL[threshold_bias] = {}
    bin_centers_final_CL[threshold_bias] = {}
    fraction_biased_CL[threshold_bias] = {}
    NN_points_CL[threshold_bias] = {}




    names_load_noCL = [
        "only_inference_models_illustris_eagle_kmax_0.6_box_3000",
        "only_inference_models_eagle_bahamas_kmax_0.6_box_3000",
        "only_inference_models_bahamas_illustris_kmax_0.6_box_3000"
    ]
    for ii, name_load in enumerate(names_load_noCL):
        y_final_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "y_final.npy"))
        bin_centers_final_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "bin_centers_final.npy"))
        fraction_biased_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "fraction_biased.npy"))
        NN_points_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "NN_points_final.npy"))
        fraction_biased_noCL[threshold_bias][name_load] = np.zeros((y_final_noCL[threshold_bias][name_load][...,-1].shape[0], y_final_noCL[threshold_bias][name_load][...,-1].shape[1]))
        mask =np.abs(bin_centers_final_noCL[threshold_bias][name_load]) > threshold_bias
        for jj in range(bin_centers_final_noCL[threshold_bias][name_load].shape[0]):
            for kk in range(bin_centers_final_noCL[threshold_bias][name_load].shape[1]):
                fraction_biased_noCL[threshold_bias][name_load][jj,kk] = np.sum(y_final_noCL[threshold_bias][name_load][jj, kk][mask[jj, kk]])/NN_points_noCL[threshold_bias][name_load][jj,kk]

    names_load_CL = [
        "only_inference_CL_Wein_models_illustris_eagle_kmax_0.6_box_3000",
        "only_inference_CL_Wein_models_eagle_bahamas_kmax_0.6_box_3000",
        "only_inference_CL_Wein_models_bahamas_illustris_kmax_0.6_box_3000"
    ]
    for ii, name_load in enumerate(names_load_CL):
        y_final_CL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "y_final.npy"))
        bin_centers_final_CL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "bin_centers_final.npy"))
        NN_points_CL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "NN_points_final.npy"))
        fraction_biased_CL[threshold_bias][name_load] = np.zeros((y_final_CL[threshold_bias][name_load][...,-1].shape[0], y_final_CL[threshold_bias][name_load][...,-1].shape[1]))
        mask =np.abs(bin_centers_final_CL[threshold_bias][name_load]) > threshold_bias
        for jj in range(bin_centers_final_CL[threshold_bias][name_load].shape[0]):
            for kk in range(bin_centers_final_CL[threshold_bias][name_load].shape[1]):
                fraction_biased_CL[threshold_bias][name_load][jj,kk] = np.sum(y_final_CL[threshold_bias][name_load][jj, kk][mask[jj, kk]])/NN_points_CL[threshold_bias][name_load][jj,kk]

    colors = ['#B54773', '#B13F3A', '#905F84']
    custom_model_names = ["illustris-eagle", "eagle-bahamas", "bahamas-illustris"]



    names_load_noCL = [
        "only_inference_models_v1v2_kmax_0.6_box_3000",
        "only_inference_models_v1v3_kmax_0.6_box_3000",
        "only_inference_models_v2v3_kmax_0.6_box_3000"
    ]
    for ii, name_load in enumerate(names_load_noCL):
        y_final_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "y_final.npy"))
        bin_centers_final_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "bin_centers_final.npy"))
        fraction_biased_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "fraction_biased.npy"))
        NN_points_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "NN_points_final.npy"))
        fraction_biased_noCL[threshold_bias][name_load] = np.zeros((y_final_noCL[threshold_bias][name_load][...,-1].shape[0], y_final_noCL[threshold_bias][name_load][...,-1].shape[1]))
        mask =np.abs(bin_centers_final_noCL[threshold_bias][name_load]) > threshold_bias
        for jj in range(bin_centers_final_noCL[threshold_bias][name_load].shape[0]):
            for kk in range(bin_centers_final_noCL[threshold_bias][name_load].shape[1]):
                fraction_biased_noCL[threshold_bias][name_load][jj,kk] = np.sum(y_final_noCL[threshold_bias][name_load][jj, kk][mask[jj, kk]])/NN_points_noCL[threshold_bias][name_load][jj,kk]

    names_load_CL = [
        "only_inference_CL_Wein_models_v1v2_kmax_0.6_box_3000",
        "only_inference_CL_Wein_models_v1v3_kmax_0.6_box_3000",
        "only_inference_CL_Wein_models_v2v3_kmax_0.6_box_3000"
    ]
    for ii, name_load in enumerate(names_load_CL):
        y_final_CL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "y_final.npy"))
        bin_centers_final_CL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "bin_centers_final.npy"))
        NN_points_CL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "NN_points_final.npy"))
        fraction_biased_CL[threshold_bias][name_load] = np.zeros((y_final_CL[threshold_bias][name_load][...,-1].shape[0], y_final_CL[threshold_bias][name_load][...,-1].shape[1]))
        mask =np.abs(bin_centers_final_CL[threshold_bias][name_load]) > threshold_bias
        for jj in range(bin_centers_final_CL[threshold_bias][name_load].shape[0]):
            for kk in range(bin_centers_final_CL[threshold_bias][name_load].shape[1]):
                fraction_biased_CL[threshold_bias][name_load][jj,kk] = np.sum(y_final_CL[threshold_bias][name_load][jj, kk][mask[jj, kk]])/NN_points_CL[threshold_bias][name_load][jj,kk]

    colors = ['#8F7B61', '#268C70', '#96901D']
    custom_model_names = ["v1-v2", "v1-v3", "v2-v3"]



    names_load_noCL = [
        "only_inference_models_f0f1_kmax_0.6_box_3000",
        "only_inference_models_f2f3_kmax_0.6_box_3000",
        "only_inference_models_f4f5_kmax_0.6_box_3000",
        "only_inference_models_f6f7_kmax_0.6_box_3000",
        "only_inference_models_f8f9_kmax_0.6_box_3000"
    ]
    for ii, name_load in enumerate(names_load_noCL):
        y_final_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "y_final.npy"))
        bin_centers_final_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "bin_centers_final.npy"))
        fraction_biased_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "fraction_biased.npy"))
        NN_points_noCL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "NN_points_final.npy"))
        fraction_biased_noCL[threshold_bias][name_load] = np.zeros((y_final_noCL[threshold_bias][name_load][...,-1].shape[0], y_final_noCL[threshold_bias][name_load][...,-1].shape[1]))
        mask =np.abs(bin_centers_final_noCL[threshold_bias][name_load]) > threshold_bias
        for jj in range(bin_centers_final_noCL[threshold_bias][name_load].shape[0]):
            for kk in range(bin_centers_final_noCL[threshold_bias][name_load].shape[1]):
                fraction_biased_noCL[threshold_bias][name_load][jj,kk] = np.sum(y_final_noCL[threshold_bias][name_load][jj, kk][mask[jj, kk]])/NN_points_noCL[threshold_bias][name_load][jj,kk]

    names_load_CL = [
        "only_inference_CL_Wein_models_f0f1_kmax_0.6_box_3000",
        "only_inference_CL_Wein_models_f2f3_kmax_0.6_box_3000",
        "only_inference_CL_Wein_models_f4f5_kmax_0.6_box_3000",
        "only_inference_CL_Wein_models_f6f7_kmax_0.6_box_3000",
        "only_inference_CL_Wein_models_f8f9_kmax_0.6_box_3000"
    ]
    for ii, name_load in enumerate(names_load_CL):
        y_final_CL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "y_final.npy"))
        bin_centers_final_CL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "bin_centers_final.npy"))
        NN_points_CL[threshold_bias][name_load] = np.load(os.path.join(path_global_load, name_load, "NN_points_final.npy"))
        fraction_biased_CL[threshold_bias][name_load] = np.zeros((y_final_CL[threshold_bias][name_load][...,-1].shape[0], y_final_CL[threshold_bias][name_load][...,-1].shape[1]))
        mask =np.abs(bin_centers_final_CL[threshold_bias][name_load]) > threshold_bias
        for jj in range(bin_centers_final_CL[threshold_bias][name_load].shape[0]):
            for kk in range(bin_centers_final_CL[threshold_bias][name_load].shape[1]):
                fraction_biased_CL[threshold_bias][name_load][jj,kk] = np.sum(y_final_CL[threshold_bias][name_load][jj, kk][mask[jj, kk]])/NN_points_CL[threshold_bias][name_load][jj,kk]

    colors = ['#0EF1FF', '#47B9FF', '#8080FF', '#BE59FF', '#F10EFF']
    custom_model_names = ["f0-f1", "f2-f3", "f4-f5", "f6-f7", "f8-f9"]

In [ ]:
threshold_bias = 3.0

In [ ]:
fontsize=26
fontsize1=18
# colors = cl_inference.plot_utils.get_N_colors(len(names_load_CL), mpl.colormaps['prism'])
custom_titles=[r'$\Omega_\mathrm{c}$', r'$\Omega_\mathrm{b}$', r'$h$', r'$n_\mathrm{s}$', r'$\sigma_{8,\mathrm{c}}$']

fig, axs = plt.subplots(
    1, len(custom_titles), figsize=(5.2*len(custom_titles), 5.2)
)
axs[0].set_ylabel(r'Normalized Counts ', size=fontsize)

custom_lines = []
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'Bias ', size=fontsize)
    ax.axvline(0, c='k', ls=':', lw=1)
    ax.axvspan(np.min(bin_centers_all[threshold_bias][ii_cosmo_param, -1]), -threshold_bias, alpha=0.2, facecolor ='red')
    ax.axvspan(threshold_bias, np.max(bin_centers_all[threshold_bias][ii_cosmo_param, -1]), alpha=0.2, facecolor ='red')

    ax.plot(
        bin_centers_all[threshold_bias][ii_cosmo_param, -1],
        y_final_all[threshold_bias][ii_cosmo_param, -1] / NN_points_all[threshold_bias][ii_cosmo_param,-1],
        color='grey', lw=2, alpha=0.9, ls='-'
    )
    
    for ii, name_load in enumerate(names_load_noCL):
        ax.plot(
            bin_centers_final_noCL[threshold_bias][name_load][ii_cosmo_param, -1],
            y_final_noCL[threshold_bias][name_load][ii_cosmo_param, -1]/NN_points_noCL[threshold_bias][name_load][ii_cosmo_param,-1],
            color=colors[ii], lw=2, alpha=0.9, ls='-'
        )
        if ii_cosmo_param==0:
            custom_lines.append(mpl.lines.Line2D([0], [0], color=colors[ii], ls='-', lw=10, marker=None, markersize=9))
            
    for ii, name_load in enumerate(names_load_CL):
        ax.plot(
            bin_centers_final_CL[threshold_bias][name_load][ii_cosmo_param, -1],
            y_final_CL[threshold_bias][name_load][ii_cosmo_param, -1]/NN_points_CL[threshold_bias][name_load][ii_cosmo_param,-1],
            color=colors[ii], lw=2, alpha=0.9, ls='--'
        )
# custom_lines.append(mpl.lines.Line2D([0], [0], color='grey', ls='-', lw=10, marker=None, markersize=9))
# custom_model_names.append("all")
legend = axs[0].legend(custom_lines, custom_model_names, loc='upper left',fancybox=True, shadow=True, ncol=1,fontsize=14)
fig.add_artist(legend)

custom_lines1=[
    mpl.lines.Line2D([0], [0], color='grey', ls='-', lw=2, marker=None, markersize=9),
    mpl.lines.Line2D([0], [0], color='grey', ls='--', lw=2, marker=None, markersize=9)
]
custom_labels1 = [
    'no CL',
    'CL'
]
legend = axs[0].legend(custom_lines1, custom_labels1, loc='upper right',fancybox=True, shadow=True, ncol=1,fontsize=14)
fig.add_artist(legend)

fig.set_tight_layout(True)

In [ ]:
fontsize=26
fontsize1=18
custom_titles=[r'$\Omega_\mathrm{c}$', r'$\Omega_\mathrm{b}$', r'$h$', r'$n_\mathrm{s}$', r'$\sigma_{8,\mathrm{c}}$']

colors = [
    '#B54773', '#B13F3A', '#905F84',
    '#8F7B61', '#268C70', '#96901D',
    '#0EF1FF', '#47B9FF', '#8080FF', '#BE59FF', '#F10EFF'
]
custom_model_names = [
    "illustris-eagle", "eagle-bahamas", "bahamas-illustris",
    "v1-v2", "v1-v3", "v2-v3",
    "f0-f1", "f2-f3", "f4-f5", "f6-f7", "f8-f9"
]

fig, axs = plt.subplots(
    1, len(custom_titles), figsize=(5.2*len(custom_titles), 5.2)
)
axs[0].set_ylabel(r'Fraction Biased ', size=fontsize)

sizes = (np.arange(len(list_threshold_bias))+1)*40
custom_lines2=[]
custom_labels2 = []
for ii_threshold, threshold_bias in enumerate(list_threshold_bias):
    custom_lines2.append(mpl.lines.Line2D([0], [0], color='grey', ls='-', lw=0, marker='o', markersize=sizes[ii_threshold]/10))
    custom_labels2.append(r'$|\sigma|_{thr}=$'+str(threshold_bias))
    for ii_cosmo_param in range(len(custom_titles)):
        ax = axs[ii_cosmo_param]
        ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
        ax.minorticks_off()

        ax.axvline(np.array([0]), color='grey', ls='--', lw=0.7)
        ax.scatter(
            np.array([0]), fraction_biased_all[threshold_bias][ii_cosmo_param, -1],
            color='grey', s=sizes[ii_threshold], marker='x'
        )

        for ii, name_load in enumerate(fraction_biased_noCL[threshold_bias].keys()):
            ax.axvline(np.array([ii+1]), color=colors[ii], ls='--', lw=0.7)
            ax.scatter(
                np.array([ii+1]), fraction_biased_noCL[threshold_bias][name_load][ii_cosmo_param, -1],
                color=colors[ii], s=sizes[ii_threshold], marker='^'
            )
        pos_list_xticks = [0]
        name_list_xticks = ["all"]
        for ii, name_load in enumerate(fraction_biased_CL[threshold_bias].keys()):
            ax.scatter(
                np.array([ii+1]), fraction_biased_CL[threshold_bias][name_load][ii_cosmo_param, -1],
                color=colors[ii], s=sizes[ii_threshold], marker='v'
            )
            pos_list_xticks.append(ii+1)
            name_list_xticks.append(custom_model_names[ii])
        ax.set_xticks(pos_list_xticks, name_list_xticks, rotation=40, fontsize=14, ha='right')
        ax.set_ylim([-0.1,1.1])
        ax.set_xlim([-0.5, len(fraction_biased_noCL[threshold_bias].keys())+0.5])
        
        
custom_lines1=[
    mpl.lines.Line2D([0], [0], color='grey', ls='-', lw=0, marker='^', markersize=9),
    mpl.lines.Line2D([0], [0], color='grey', ls='-', lw=0, marker='v', markersize=9)
]
custom_labels1 = [
    'no CL',
    'CL'
]
legend = axs[0].legend(custom_lines1, custom_labels1, loc='upper left',fancybox=True, shadow=True, ncol=1,fontsize=14)
fig.add_artist(legend)

legend = axs[0].legend(custom_lines2, custom_labels2, loc='upper right',fancybox=True, shadow=True, ncol=1,fontsize=14)
fig.add_artist(legend)
        
        
fig.set_tight_layout(True)